### Conexion a base de datos para btener los usuarios que tenemos guardados

In [ ]:
#Conexión a la base de datos
from pymongo import MongoClient
client = MongoClient()
client = MongoClient("localhost",27017)
client = MongoClient("mongodb://localhost:27017/")
db = client["proyectodb"]
db.collection_names()

In [ ]:
#Consumo del servicio de Twitter
import tweepy
auth = tweepy.OAuthHandler('n5Vo0aNUy4l2eu52cwJicUJ33', 'guPfeOe6axjD9uOeSLLGC8IfClswpiLX5tHafdNNtn4FErKmPi')
auth.set_access_token('784154913454645248-HefUzw6JOeMibIRLsNNu7hESr77wzCy', 'BHtr8sJrbwhbUapFE4NtsHL3OncTJ4etAaUydKOktWLV3')
api = tweepy.API(auth)

In [ ]:
#Colecciones de la base de datos
listaMujeresTest = api.list_members(slug='JimesMujeres',owner_screen_name='@jimesproject', count=2000)
listaHombresTest = api.list_members(slug='JimesHombres',owner_screen_name='@jimesproject', count=2000)


collHombres= db.usuarios_hombres.find();

collMujeres= db.usuarios_mujeres.find();

collHombresConcat = db['tweets_hombres_concat_url_cambiada']

collMujeresConcat = db['tweets_mujeres_concat_url_cambiada']

collHombresConcatNuestros = db['tweets_hombres_concat_nuestros']

collMujeresConcatNuestros = db['tweets_mujeres_concat_nuestros']

collHombresFind = db['usuarios_hombres']

collMujeresFind = db['usuarios_mujeres']

collFotoPerfilHombres = db['foto_perfil_hombres']

collFotoPerfilMujeres = db['foto_perfil_mujeres']

collFotoPerfilHombresNuestro = db['foto_perfil_hombres_nuestro']

collFotoPerfilMujeresNuestro = db['foto_perfil_mujeres_nuestro']

collEmojisHombres = db['emojis_hombres']

collEmojisMujeres = db['emojis_mujeres']

collEmojisHombresNuestros = db['emojis_hombres_nuestros']

collEmojisMujeresNuestros = db['emojis_mujeres_nuestros']


In [ ]:
#Punteros auxiliares para las colecciones
collHombresAuxN = collHombres

collMujeresAuxN = collMujeres

## Usuarios de Entrenamiento

In [ ]:
#Armamos lista de usuarios
US_train = []
vClasifUs = []
for i in range(0,7000):
    try:
        US_train = US_train + [collHombresAux[i]['screen_name']]
        vClasifUs = vClasifUs + [0]
        US_train = US_train + [collMujeresAux[i]['screen_name']]
        vClasifUs = vClasifUs + [1]
    except Exception as e:
        print(e)


## Dividimos el conjunto de usuarios

In [ ]:
#Librerias
import numpy as np
import re
from pprint import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics   
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.model_selection import train_test_split

In [ ]:
#Reducción de atributos para cada clasificador
vectorizer = TfidfVectorizer()
vectorizerLinear1 = TfidfVectorizer(max_df=0.6)
vectorizerNB1 = TfidfVectorizer(min_df=0.01)
vectorizerDT_Kmeans = TfidfVectorizer(max_df=0.5,min_df=0.05)
vectorizerMLP1 = TfidfVectorizer(max_df=0.5, min_df=0.04)

## Se dividen los tweets y se agrega la descripción

In [ ]:
#Usuarios de entrenamiento
X_tweetsTrain = []
X_tweetsTest = []
y_vClasifTweetsTrain = vClasifUs

for us in US_train:
    try:   
        cursor = collHombresConcat.find({"screen_name":{"$in":[us]}})
        cursorUsuario = collHombresFind.find({"screen_name":{"$in":[us]}})
        if cursor.count() == 0:
            cursor = collMujeresConcat.find({"screen_name":{"$in":[us]}})
            cursorUsuario = collMujeresFind.find({"screen_name":{"$in":[us]}})
        if cursor.count() > 1:
            print(us)    
        for s in cursor:
            X_tweetsTrain = X_tweetsTrain + [s['text']  + cursorUsuario[0]['description']]        
    except Exception as e:
        print(e)          


In [ ]:
#Usuarios de test
US_test =[]
y_vClasifTweetsTest=[]
X_tweetsTest = []
cont=0
for i in range(0,161):
    try:   
        cursorH = collHombresConcatNuestros.find({"screen_name":{"$in":[listaHombresTest[i].screen_name]}})
        US_test+=[listaHombresTest[i].screen_name]
        
        if(cursorH.count() == 0):
            print(listaHombresTest[i].screen_name)
        y_vClasifTweetsTest+=[0]
        for s in cursorH:#aca siempre hay 1
                X_tweetsTest = X_tweetsTest + [s['text'] + listaHombresTest[i].description] 
                cont+=1
                
                #print(i)
        if (i < 79):
            cursorM = collMujeresConcatNuestros.find({"screen_name":{"$in":[listaMujeresTest[i].screen_name]}})
            if(cursorM.count() == 0):
                print(listaMujeresTest[i].screen_name)
            US_test+=[listaMujeresTest[i].screen_name]
            y_vClasifTweetsTest+=[1]
            for s in cursorM:#aca siempre hay 1
                X_tweetsTest = X_tweetsTest + [s['text'] + listaMujeresTest[i].description] 
                cont+=1
    
    except Exception as e:
        print(e)          
        

### Transformamos los tweets para entrenamiento y test

In [ ]:
#vX_train = vectorizerLinear1.fit_transform(X_tweetsTrain)
#vX_test = vectorizerLinear1.transform(X_tweetsTest)

vX_train = vectorizerMLP1.fit_transform(X_tweetsTrain)
vX_test = vectorizerMLP1.transform(X_tweetsTest)

#vX_train = vectorizerNB1.fit_transform(X_tweetsTrain)
#vX_test = vectorizerNB1.transform(X_tweetsTest)

#vX_train = vectorizerDT_Kmeans_RF.fit_transform(X_tweetsTrain)
#vX_test = vectorizerDT_Kmeans_RF.transform(X_tweetsTest)


#vX_train = vectorizer.fit_transform(X_tweetsTrain)
#vX_test = vectorizer.transform(X_tweetsTest)

## Seleccionar los emoticones que se encuentran dentro de un umbral

In [ ]:
from collections import Counter
import emoji

def obtenerNombreEmoji(stringDemojize):
    separador= ":"
    occurrences = stringDemojize.count(separador)
    indices = [i for i, a in enumerate(stringDemojize) if a == separador]
    return stringDemojize[indices[0]+1 : indices[1]]

def obtenerKEmojisMasUsadosTrain(threshold):
    resEmojis = {}
    contResEmojis = 0
    colEmojisHombresCursor = db.emojis_hombres.find()
    colPerfilMujeresCursor = db.emojis_mujeres.find()
    colEmojisHombresCursorAux = colEmojisHombresCursor
    colPerfilMujeresCursorAux = colPerfilMujeresCursor
    datos_basicos = {}

    for jsonEmoji in colEmojisHombresCursorAux:
        for e in jsonEmoji['emojis']: 
            Nomemoji = obtenerNombreEmoji(emoji.demojize(e))
            if(not (Nomemoji in datos_basicos)):
                datos_basicos[Nomemoji] = 1
            else:
                datos_basicos[Nomemoji] = datos_basicos[Nomemoji] + 1

    for jsonFoto in colPerfilMujeresCursorAux:
        for  e in jsonEmoji['emojis']: 
            Nomemoji = obtenerNombreEmoji(emoji.demojize(e))
            if(not (Nomemoji in datos_basicos)):
                datos_basicos[Nomemoji] = 1
            else:
                datos_basicos[Nomemoji] = datos_basicos[Nomemoji] + 1
    arregloTags = {}
    
    for k in datos_basicos:
        if datos_basicos[k] > threshold:  
            resEmojis[k] = contResEmojis
            contResEmojis = contResEmojis + 1
    return resEmojis

## Seleccionar los tags de la foto de perfil que se encuentran dentro de un umbral

In [ ]:
from collections import Counter

def obtenerKTagsMasUsadosTrain(threshold):
    resTags = {}
    contResTags = 0
    colPerfilHombresCursor = db.foto_perfil_hombres.find()
    colPerfilMujeresCursor = db.foto_perfil_mujeres.find()
    colPerfilHombresCursorAux = colPerfilHombresCursor
    colPerfilMujeresCursorAux = colPerfilMujeresCursor
    datos_basicos = {}
    for jsonFoto in colPerfilHombresCursorAux:
        for t in jsonFoto['json_imagen']['tags']:
            if(not (t['name'] in datos_basicos)):
                datos_basicos[t['name']] = 1
            else:
                datos_basicos[t['name']] = datos_basicos[t['name']] + 1

    for jsonFoto in colPerfilMujeresCursorAux:
        for t in jsonFoto['json_imagen']['tags']:
            if(not (t['name'] in datos_basicos)):
                datos_basicos[t['name']] = 1
            else:
                datos_basicos[t['name']] = datos_basicos[t['name']] + 1
    arregloTags = {}
    
    for k in datos_basicos:
        if datos_basicos[k] > threshold:  
            resTags[k] = contResTags
            contResTags = contResTags + 1
    return resTags

##  Seleccionar las categorias de la foto de perfil que se encuentran dentro de un umbral

In [ ]:
from collections import Counter

def obtenerKCatsMasUsadosTrain(threshold):
    resCategories = {}
    contResCats = 0
    colPerfilHombresCursor = db.foto_perfil_hombres.find()
    colPerfilMujeresCursor = db.foto_perfil_mujeres.find()
    colPerfilHombresCursorAux = colPerfilHombresCursor
    colPerfilMujeresCursorAux = colPerfilMujeresCursor
    datos_basicos = {}
    cont = 0
    for jsonFoto in colPerfilHombresCursorAux:
        try: 
            for t in jsonFoto['json_imagen']['categories']:
                if(not (t['name'] in datos_basicos)):
                    datos_basicos[t['name']] = 1
                else:
                    datos_basicos[t['name']] = datos_basicos[t['name']] + 1
        except Exception as e:
            cont = cont +1 
            
    for jsonFoto in colPerfilMujeresCursorAux:
        try:
            for t in jsonFoto['json_imagen']['categories']:
                if(not (t['name'] in datos_basicos)):
                    datos_basicos[t['name']] = 1
                else:
                    datos_basicos[t['name']] = datos_basicos[t['name']] + 1
        except Exception as e:
            cont = cont +1 

            
    for k in datos_basicos:
        if datos_basicos[k] > threshold:  
            resCategories[k] = contResCats
            contResCats = contResCats + 1
    return resCategories

## Setear valores para armar las matrices de emojis y de imagenes (con tags y cats)

In [ ]:
#Seleccionar las etiquetas y categorias a considerar
tagsSeleccionadosTrain = obtenerKTagsMasUsadosTrain(0)
catsSeleccionadosTrain = obtenerKCatsMasUsadosTrain(0)

#Seleccionar los emoticones
emojisSeleccionadosTrain = obtenerKEmojisMasUsadosTrain(0)

## Definimos las matrices de los emoticones y las imágenes

In [ ]:
#IMAGENES
matrizTagsTrain = np.zeros((len(US_train), len(tagsSeleccionadosTrain)))
matrizTagsTest =  np.zeros((len(US_test), len(tagsSeleccionadosTrain)))

matrizCatsTrain = np.zeros((len(US_train), len(catsSeleccionadosTrain)))
matrizCatsTest =  np.zeros((len(US_test), len(catsSeleccionadosTrain)))

#EMOTICONES
matrizEmojisTrain = np.zeros((len(US_train), len(emojisSeleccionadosTrain)))
matrizEmojisTest =  np.zeros((len(US_test), len(emojisSeleccionadosTrain)))

## Armar matrices imagen tags y cats + emojis

## Se buscan los emoticones de los usuarios que estan en los conjuntos US_train y US_test


In [ ]:
contFilaTrain = 0
for us in US_train:
    try:   
        cursor = collEmojisHombres.find({"screen_name":{"$in":[us]}})
        if cursor.count() == 0:
            cursor = collEmojisMujeres.find({"screen_name":{"$in":[us]}})      
        for jsonEmoji in cursor:
            for e in jsonEmoji['emojis']: 
                nomemoji = obtenerNombreEmoji(emoji.demojize(e))
                if(nomemoji in emojisSeleccionadosTrain):
                    matrizEmojisTrain.itemset((contFilaTrain,emojisSeleccionadosTrain[nomemoji]),1) 
            contFilaTrain = contFilaTrain +1

                
    except Exception as e:
        print(e)      

In [ ]:
contFilaTest = 0

for us in US_test:
    try:   
        cursor = collEmojisHombresNuestros.find({"screen_name":{"$in":[us]}})
        if cursor.count() == 0:
            cursor = collEmojisMujeresNuestros.find({"screen_name":{"$in":[us]}}) 
        for jsonEmoji in cursor:
            for e in jsonEmoji['emojis']: 
                nomemoji = obtenerNombreEmoji(emoji.demojize(e))
                print(nomemoji)
                if(nomemoji in emojisSeleccionadosTrain):
                    matrizEmojisTest.itemset((contFilaTest,emojisSeleccionadosTrain[nomemoji]),1) 
            contFilaTest = contFilaTest +1      
    except Exception as e:
        print(e)   


### Se buscan las etiquetas de las imagenes de los usuarios que estan en los conjuntos US_train y US_test

In [ ]:
yUs_train = vClasifUs
yUs_test = y_vClasifTweetsTest

y_vClasifImageTrain = yUs_train
y_vClasifImageTest = yUs_test

contFilaTrain = 0
for us in US_train:
    try:   
        cursor = collFotoPerfilHombres.find({"screen_name":{"$in":[us]}})
        if cursor.count() == 0:
            cursor = collFotoPerfilMujeres.find({"screen_name":{"$in":[us]}})      
        for s in cursor: 
            for t in s['json_imagen']['tags']:
                if(t['name'] in tagsSeleccionadosTrain):
                    matrizTagsTrain.itemset((contFilaTrain,tagsSeleccionadosTrain[t['name']]),1) 
            contFilaTrain = contFilaTrain +1
                
                
    except Exception as e:
        print(e)          


In [ ]:
contFilaTest = 0

for us in US_test:
    try:   
        cursor = collFotoPerfilHombresNuestro.find({"screen_name":{"$in":[us]}})
        if cursor.count() == 0:
            cursor = collFotoPerfilMujeresNuestro.find({"screen_name":{"$in":[us]}})  
        for s in cursor:
            for t in s['json_imagen']['tags']:
                if(t['name'] in tagsSeleccionadosTrain):
                    matrizTagsTest.itemset((contFilaTest,tagsSeleccionadosTrain[t['name']]),1)
            contFilaTest = contFilaTest +1      
    except Exception as e:
        print(e)   

### Se buscan las cetegorias de las imagenes de los usuarios que estan en los conjuntos US_train y US_test

In [ ]:
y_vClasifImageTrain = yUs_train
y_vClasifImageTest = yUs_test

contFilaTrain = 0
for us in US_train:
    try:   
        cursor = collFotoPerfilHombres.find({"screen_name":{"$in":[us]}})
        if cursor.count() == 0:
            cursor = collFotoPerfilMujeres.find({"screen_name":{"$in":[us]}})      
        for s in cursor: 
            for t in s['json_imagen']['categories']:
                if(t['name'] in catsSeleccionadosTrain):
                    matrizCatsTrain.itemset((contFilaTrain,catsSeleccionadosTrain[t['name']]),1)
            contFilaTrain = contFilaTrain +1          
    except Exception as e:
        print(e)          


In [ ]:
contFilaTest = 0

for us in US_test:
    try:   
        cursor = collFotoPerfilHombresNuestro.find({"screen_name":{"$in":[us]}})
        if cursor.count() == 0:
            cursor = collFotoPerfilMujeresNuestro.find({"screen_name":{"$in":[us]}})
            if cursor.count() == 0:
                print(us)
        for s in cursor:
            for t in s['json_imagen']['categories']:
                if(t['name'] in catsSeleccionadosTrain):
                    matrizCatsTest.itemset((contFilaTest,catsSeleccionadosTrain[t['name']]),1) #ver de obtener el confidence
            contFilaTest = contFilaTest +1      
    except Exception as e:
        print(e)   

In [ ]:
#Concatenación de matrices - Armado de matriz final: Input para los clasificadores

from scipy import sparse

c = sparse.hstack((vX_test, matrizCatsTest))
vX_test = c

In [ ]:
c = sparse.hstack((vX_test, matrizTagsTest))
vX_test = c

In [ ]:
c = sparse.hstack((vX_test, matrizEmojisTest))
vX_test = c


In [ ]:
c = sparse.hstack((vX_train, matrizCatsTrain))
vX_train = c

In [ ]:
c = sparse.hstack((vX_train, matrizTagsTrain))
vX_train = c

In [ ]:
c = sparse.hstack((vX_train, matrizEmojisTrain))
vX_train = c

## Función para imprimir performance de un clasificador

In [ ]:
#Prediccion

def performance_clasificador (vX_test, y_test, clf, avg, label): 
    yp = clf.predict(vX_test)
    #Medida de Accuracy
    print ("Accuracy:\t\t",metrics.accuracy_score(y_test,yp))
    #Medida de Precision
    print ("Precision:\t\t",metrics.precision_score(y_test,yp,average=avg, pos_label=label))
    #Medida de Recall
    print ("Recall:\t\t\t",metrics.recall_score(y_test,yp,average=avg, pos_label=label))
    #Medida F1
    print ("Medida F:\t\t",metrics.f1_score(y_test,yp,average=avg, pos_label=label))  
    #Matriz de Confusion
    print ("Matriz de Confusion:\t") 
    print (metrics.confusion_matrix(y_test,yp))
    return       

In [ ]:
#Clasificador LinearSVC ##Mejor Config
t1=time.time()
clf_svm = svm.LinearSVC(C=0.85,penalty='l2', loss='hinge',max_iter=40)
clf_svm.fit(vX_train, yUs_train)
t2=time.time()
performance_clasificador (vX_test, yUs_test, clf_svm, 'binary', 0)
performance_clasificador (vX_test, yUs_test, clf_svm, 'binary', 1)
tfinal= t2 - t1
print(tfinal)

In [ ]:
#Clasificador Naive Bayes ##Mejor Config
t1=time.time()
clf_NB = MultinomialNB(alpha=0.03).fit(vX_train, yUs_train)
t2=time.time()
performance_clasificador (vX_test, yUs_test, clf_NB, 'binary',0)
performance_clasificador (vX_test, yUs_test, clf_NB, 'binary',1)
tfinal= t2 - t1
print(tfinal)

In [ ]:
#Clasificador Árbol Decisión ##Mejor Config
t1=time.time()
clf_DT = DecisionTreeClassifier(criterion='entropy', min_samples_split=1500).fit(vX_train, yUs_train)
t2=time.time()
performance_clasificador (vX_test, yUs_test, clf_DT, 'binary',0)
performance_clasificador (vX_test, yUs_test, clf_DT, 'binary',1)
tfinal= t2 - t1
print(tfinal)

In [ ]:
#Clasificador RNA ##Mejor Config
from sklearn.neural_network import MLPClassifier
t1=time.time()
clf_NN = MLPClassifier(early_stopping=True, hidden_layer_sizes=(20,20)).fit(vX_train, yUs_train)
t2=time.time()
performance_clasificador (vX_test, yUs_test, clf_NN, 'binary',0)
performance_clasificador (vX_test, yUs_test, clf_NN, 'binary',1)
tfinal= t2 - t1
print(tfinal)

In [ ]:
#Clasificador Clustering ##Mejor Config
from sklearn.cluster import KMeans
import numpy as np

t1=time.time()
kmeans = KMeans(n_clusters=2, random_state=0).fit(vX_train)
print(kmeans.labels_)
t2=time.time()
performance_clasificador (vX_test, yUs_test, kmeans, 'binary',0)
performance_clasificador (vX_test, yUs_test, kmeans, 'binary',1)
tfinal= t2 - t1
print(tfinal)
print (kmeans.cluster_centers_)


In [ ]:
#Clasificador Bosuqe Aleatorios ##Mejor Config
t1=time.time()
clf = RandomForestClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=1500, min_weight_fraction_leaf=0.0,
            random_state=None, n_jobs=-1,oob_score=True, warm_start=True).fit(vX_train, yUs_train)
t2=time.time()
performance_clasificador (vX_test, yUs_test, clf, 'binary',0)
performance_clasificador (vX_test, yUs_test, clf, 'binary',1)
tfinal= t2 - t1
print(tfinal)

In [ ]:
#Matriz de confusión para el mejor resultado: MLPClassifier
yp = clf_NN.predict(vX_test)
cm=metrics.confusion_matrix(yUs_test,yp)

In [ ]:
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
#class_names =[0,1]
class_names =['H','M']
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cm, classes=class_names,
                      title='Matriz de Confusión C4')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cm, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()